In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from ultralytics import YOLO
from sklearn.metrics import roc_auc_score
import numpy as np
from pathlib import Path
import yaml
from tqdm import tqdm
import math

# =========================================================
# 설정
# =========================================================
ENGINE = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251020/weights/best_fp16_bs1_640px_static.engine"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251017/data.yaml"

CONF = 0.5
IOU = 0.5

# =========================================================
# 모델 로드
# =========================================================
model = YOLO(ENGINE)
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
classes = data_cfg["names"]
num_classes = len(classes)

# =========================================================
# 1️⃣ YOLO val() 평가
# =========================================================
print("🚀 Running YOLO val() for mAP metrics...")
val_results = model.val(
    data=DATA_YAML,
    split="test",
    imgsz=640,
    batch=1,
    rect=False,
    device=0,
    deterministic=True,
    seed=1,
    conf=CONF,
    iou=IOU,
    augment=False,
    agnostic_nms=True,
)

precision_mean = val_results.box.mp
recall_mean = val_results.box.mr
map50 = val_results.box.map50
map5095 = val_results.box.map

print("\n📊 YOLO Validation Summary")
print("──────────────────────────────────────────────")
print(f"Precision(mean): {precision_mean:.3f}")
print(f"Recall(mean):    {recall_mean:.3f}")
print(f"mAP50:           {map50:.3f}")
print(f"mAP50-95:        {map5095:.3f}")

# --- per-class metric 저장 (Confused_Object 점프 방식)
per_class_metrics = []
for i, name in enumerate(classes):
    if i == 3:  # Confused_Object (test에 없음)
        per_class_metrics.append((0.0, 0.0, 0.0, 0.0))
        continue

    idx = i if i < 3 else i - 1  # 이후 클래스는 1칸 당김
    if idx >= len(val_results.box.p):
        per_class_metrics.append((0.0, 0.0, 0.0, 0.0))
    else:
        p_i, r_i, ap50_i, ap5095_i = val_results.box.class_result(idx)
        per_class_metrics.append((p_i, r_i, ap50_i, ap5095_i))

# =========================================================
# 2️⃣ 예측 수행 (시각화 저장 포함)
# =========================================================
print("\n🚀 Running YOLO predict() for AUROC calculation + visualization...")
test_img_dir = Path(data_cfg["path"]) / "images" / "test"
test_label_dir = Path(data_cfg["path"]) / "labels" / "test"

save_dir = Path("runs/predict_auroc_visual")
save_dir.mkdir(parents=True, exist_ok=True)

results = model.predict(
    source=str(test_img_dir),
    imgsz=640,
    conf=CONF,
    iou=IOU,
    agnostic_nms=True,
    rect=False,
    device=0,
    save=True,
    save_txt=True,
    project=str(save_dir.parent),
    name=save_dir.name,
    exist_ok=True
)

# =========================================================
# IoU 계산 함수
# =========================================================
def box_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0

# =========================================================
# 3️⃣ 매칭 및 통계 계산
# =========================================================
per_class_true = {i: [] for i in range(num_classes)}
per_class_score = {i: [] for i in range(num_classes)}
gt_counts = np.zeros(num_classes, dtype=int)
tp_counts = np.zeros(num_classes, dtype=int)
fp_counts = np.zeros(num_classes, dtype=int)
fn_counts = np.zeros(num_classes, dtype=int)

for r in tqdm(results, desc="Evaluating AUROC per class"):
    name = Path(r.path).stem
    label_file = test_label_dir / f"{name}.txt"

    gt_boxes, gt_classes = [], []
    if label_file.exists():
        with open(label_file, "r") as f:
            for line in f:
                cls, x, y, w, h = map(float, line.split())
                cls = int(cls)
                h_img, w_img = r.orig_shape
                x1, y1 = (x - w/2)*w_img, (y - h/2)*h_img
                x2, y2 = (x + w/2)*w_img, (y + h/2)*h_img
                gt_boxes.append([x1, y1, x2, y2])
                gt_classes.append(cls)
                gt_counts[cls] += 1

    gt_boxes = np.array(gt_boxes)
    gt_classes = np.array(gt_classes)

    if len(r.boxes) == 0:
        for c in gt_classes:
            fn_counts[c] += 1
        continue

    pred_boxes = r.boxes.xyxy.cpu().numpy()
    pred_confs = r.boxes.conf.cpu().numpy()
    pred_classes = r.boxes.cls.cpu().numpy().astype(int)

    used_gt = set()
    for pred_box, pred_conf, pred_cls in zip(pred_boxes, pred_confs, pred_classes):
        same_cls_idx = np.where(gt_classes == pred_cls)[0]
        gt_same_cls = gt_boxes[same_cls_idx]
        iou_max, best_gt_idx = 0, -1
        if len(gt_same_cls) > 0:
            for j, gt_box in enumerate(gt_same_cls):
                iou_val = box_iou(pred_box, gt_box)
                if iou_val > iou_max:
                    iou_max = iou_val
                    best_gt_idx = same_cls_idx[j]
        label = 1 if (iou_max >= IOU and best_gt_idx not in used_gt) else 0
        per_class_true[pred_cls].append(label)
        per_class_score[pred_cls].append(pred_conf)
        if label == 1:
            used_gt.add(best_gt_idx)
            tp_counts[pred_cls] += 1
        else:
            fp_counts[pred_cls] += 1

    for i, gt_c in enumerate(gt_classes):
        if i not in used_gt:
            fn_counts[gt_c] += 1

# =========================================================
# 4️⃣ AUROC + YOLO VAL Metrics 출력
# =========================================================
print("\n📈 AUROC + YOLO VAL Metrics per class")
print("────────────────────────────────────────────────────────────────────────────")
aurocs = []

for i, name in enumerate(classes):
    y_true = np.array(per_class_true[i])
    y_score = np.array(per_class_score[i])
    p_i, r_i, m50_i, m5095_i = per_class_metrics[i]

    if gt_counts[i] == 0 or len(y_true) < 2 or len(set(y_true)) < 2:
        auroc = float('nan')
    else:
        try:
            auroc = roc_auc_score(y_true, y_score)
        except ValueError:
            auroc = float('nan')

    aurocs.append(auroc)
    auroc_str = f"{auroc:.4f}" if not math.isnan(auroc) else "NaN"
    print(f"{name:<18s}: AUROC={auroc_str:<8s} | "
          f"TP={tp_counts[i]:<4d} FP={fp_counts[i]:<4d} FN={fn_counts[i]:<4d} GT={gt_counts[i]:<4d} | "
          f"P={p_i:.3f} R={r_i:.3f} mAP50={m50_i:.3f} mAP50-95={m5095_i:.3f}")

# =========================================================
# 5️⃣ 전체 요약
# =========================================================
valid_aurocs = [a for a in aurocs if not math.isnan(a)]
mean_auroc = np.mean(valid_aurocs) if valid_aurocs else float('nan')

print("────────────────────────────────────────────────────────────────────────────")
print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}" if not math.isnan(mean_auroc) else "🔹 Mean AUROC: NaN")

print("\n📊 Final Summary (Detection + AUROC)")
print("===================================================")
print(f"Precision(mean): {precision_mean:.3f}")
print(f"Recall(mean):    {recall_mean:.3f}")
print(f"mAP50:           {map50:.3f}")
print(f"mAP50-95:        {map5095:.3f}")
print(f"Mean AUROC:      {mean_auroc:.3f}")
print("---------------------------------------------------")
for i, name in enumerate(classes):
    print(f"  {name:<18s} TP={tp_counts[i]:<4d} FP={fp_counts[i]:<4d} FN={fn_counts[i]:<4d} GT={gt_counts[i]:<4d}")
print("===================================================")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
🚀 Running YOLO val() for mAP metrics...
Ultralytics 8.3.202 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Loading /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251020/weights/best_fp16_bs1_640px_static.engine for TensorRT inference...
[10/23/2025-12:47:55] [TRT] [I] Loaded engine size: 24 MiB
[10/23/2025-12:47:55] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +17, now: CPU 0, GPU 38 (MiB)
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 10129.9±771.7 MB/s, size: 1683.7 KB)
val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251017/labels/test.cache... 492 images, 10 backgrounds, 0 corrupt

Evaluating AUROC per class: 100%|██████████| 502/502 [00:00<00:00, 5969.61it/s]


📈 AUROC + YOLO VAL Metrics per class
────────────────────────────────────────────────────────────────────────────
Divot             : AUROC=0.8098   | TP=502  FP=109  FN=173  GT=675  | P=0.822 R=0.744 mAP50=0.813 mAP50-95=0.447
Fixed_Divot       : AUROC=0.7977   | TP=674  FP=73   FN=264  GT=938  | P=0.902 R=0.719 mAP50=0.825 mAP50-95=0.484
Diseased_Grass    : AUROC=NaN      | TP=22   FP=0    FN=44   GT=66   | P=1.000 R=0.333 mAP50=0.667 mAP50-95=0.444
Confused_Object   : AUROC=NaN      | TP=0    FP=0    FN=0    GT=0    | P=0.000 R=0.000 mAP50=0.000 mAP50-95=0.000
Pole              : AUROC=NaN      | TP=9    FP=0    FN=2    GT=11   | P=1.000 R=0.818 mAP50=0.909 mAP50-95=0.641
Sprinkler         : AUROC=1.0000   | TP=6    FP=1    FN=2    GT=8    | P=0.857 R=0.750 mAP50=0.856 mAP50-95=0.452
Drain             : AUROC=NaN      | TP=48   FP=0    FN=0    GT=48   | P=1.000 R=1.000 mAP50=0.995 mAP50-95=0.708
Golf_ball         : AUROC=NaN      | TP=21   FP=0    FN=0    GT=21   | P=1.000 R=1.000 

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from ultralytics import YOLO
from sklearn.metrics import roc_auc_score
import numpy as np
from pathlib import Path
import yaml
from tqdm import tqdm
import math

# =========================================================
# 설정
# =========================================================
ENGINE = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251020/weights/best_fp16_bs1_640px_static.engine"
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_test/test_20251017/data.yaml"

CONF = 0.5
IOU = 0.5

# =========================================================
# 모델 로드
# =========================================================
model = YOLO(ENGINE)
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
classes = data_cfg["names"]
num_classes = len(classes)

# =========================================================
# 1️⃣ 예측 수행
# =========================================================
print("🚀 Running YOLO predict() for AUROC calculation...")
test_img_dir = Path(data_cfg["path"]) / "images" / "test"
test_label_dir = Path(data_cfg["path"]) / "labels" / "test"

save_dir = Path("runs/predict_auroc_visual")
save_dir.mkdir(parents=True, exist_ok=True)

results = model.predict(
    source=str(test_img_dir),
    imgsz=640,
    conf=CONF,
    iou=IOU,
    agnostic_nms=True,
    rect=False,
    device=0,
    save=False,       # 시각화 비활성화
    save_txt=False,   # txt 저장 비활성화
    project=str(save_dir.parent),
    name=save_dir.name,
    exist_ok=True
)

# =========================================================
# IoU 계산 함수
# =========================================================
def box_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0

# =========================================================
# 2️⃣ 매칭 및 통계 계산 (AUROC용)
# =========================================================
per_class_true = {i: [] for i in range(num_classes)}
per_class_score = {i: [] for i in range(num_classes)}
gt_counts = np.zeros(num_classes, dtype=int)
tp_counts = np.zeros(num_classes, dtype=int)
fp_counts = np.zeros(num_classes, dtype=int)
fn_counts = np.zeros(num_classes, dtype=int)

for r in tqdm(results, desc="Evaluating AUROC per class"):
    name = Path(r.path).stem
    label_file = test_label_dir / f"{name}.txt"

    gt_boxes, gt_classes = [], []
    if label_file.exists():
        with open(label_file, "r") as f:
            for line in f:
                cls, x, y, w, h = map(float, line.split())
                cls = int(cls)
                h_img, w_img = r.orig_shape
                x1, y1 = (x - w/2)*w_img, (y - h/2)*h_img
                x2, y2 = (x + w/2)*w_img, (y + h/2)*h_img
                gt_boxes.append([x1, y1, x2, y2])
                gt_classes.append(cls)
                gt_counts[cls] += 1

    gt_boxes = np.array(gt_boxes)
    gt_classes = np.array(gt_classes)

    if len(r.boxes) == 0:
        for c in gt_classes:
            fn_counts[c] += 1
        continue

    pred_boxes = r.boxes.xyxy.cpu().numpy()
    pred_confs = r.boxes.conf.cpu().numpy()
    pred_classes = r.boxes.cls.cpu().numpy().astype(int)

    used_gt = set()
    for pred_box, pred_conf, pred_cls in zip(pred_boxes, pred_confs, pred_classes):
        same_cls_idx = np.where(gt_classes == pred_cls)[0]
        gt_same_cls = gt_boxes[same_cls_idx]
        iou_max, best_gt_idx = 0, -1
        if len(gt_same_cls) > 0:
            for j, gt_box in enumerate(gt_same_cls):
                iou_val = box_iou(pred_box, gt_box)
                if iou_val > iou_max:
                    iou_max = iou_val
                    best_gt_idx = same_cls_idx[j]
        label = 1 if (iou_max >= IOU and best_gt_idx not in used_gt) else 0
        per_class_true[pred_cls].append(label)
        per_class_score[pred_cls].append(pred_conf)
        if label == 1:
            used_gt.add(best_gt_idx)
            tp_counts[pred_cls] += 1
        else:
            fp_counts[pred_cls] += 1

    for i, gt_c in enumerate(gt_classes):
        if i not in used_gt:
            fn_counts[gt_c] += 1

# =========================================================
# 3️⃣ AUROC 계산 및 출력
# =========================================================
print("\n📈 AUROC Evaluation per class")
print("─────────────────────────────────────────────────────────────")
aurocs = []

for i, name in enumerate(classes):
    y_true = np.array(per_class_true[i])
    y_score = np.array(per_class_score[i])

    if gt_counts[i] == 0 or len(y_true) < 2 or len(set(y_true)) < 2:
        auroc = float('nan')
    else:
        try:
            auroc = roc_auc_score(y_true, y_score)
        except ValueError:
            auroc = float('nan')

    aurocs.append(auroc)
    auroc_str = f"{auroc:.4f}" if not math.isnan(auroc) else "NaN"
    print(f"{name:<18s}: AUROC={auroc_str:<8s} | "
          f"TP={tp_counts[i]:<4d} FP={fp_counts[i]:<4d} FN={fn_counts[i]:<4d} GT={gt_counts[i]:<4d}")

# =========================================================
# 4️⃣ 전체 요약
# =========================================================
valid_aurocs = [a for a in aurocs if not math.isnan(a)]
mean_auroc = np.mean(valid_aurocs) if valid_aurocs else float('nan')

print("─────────────────────────────────────────────────────────────")
if not math.isnan(mean_auroc):
    print(f"🔹 Mean AUROC (macro): {mean_auroc:.4f}")
else:
    print("🔹 Mean AUROC: NaN")
print("=============================================================")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
🚀 Running YOLO predict() for AUROC calculation...
Loading /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251020/weights/best_fp16_bs1_640px_static.engine for TensorRT inference...
[10/23/2025-12:51:11] [TRT] [W] WARNING The logger passed into createInferRuntime differs from one already registered for an existing builder, runtime, or refitter. So the current new logger is ignored, and TensorRT will use the existing one which is returned by nvinfer1::getLogger() instead.
[10/23/2025-12:51:11] [TRT] [I] Loaded engine size: 24 MiB
[10/23/2025-12:51:11] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +17, now: CPU 2, GPU 154 (MiB)

image 1/502 /home/dw/ws_job_msislab/Golf_Project/data/for_test/tes

Evaluating AUROC per class: 100%|██████████| 502/502 [00:00<00:00, 5357.57it/s]


📈 AUROC Evaluation per class
─────────────────────────────────────────────────────────────
Divot             : AUROC=0.8098   | TP=502  FP=109  FN=173  GT=675 
Fixed_Divot       : AUROC=0.7977   | TP=674  FP=73   FN=264  GT=938 
Diseased_Grass    : AUROC=NaN      | TP=22   FP=0    FN=44   GT=66  
Confused_Object   : AUROC=NaN      | TP=0    FP=0    FN=0    GT=0   
Pole              : AUROC=NaN      | TP=9    FP=0    FN=2    GT=11  
Sprinkler         : AUROC=1.0000   | TP=6    FP=1    FN=2    GT=8   
Drain             : AUROC=NaN      | TP=48   FP=0    FN=0    GT=48  
Golf_ball         : AUROC=NaN      | TP=21   FP=0    FN=0    GT=21  
─────────────────────────────────────────────────────────────
🔹 Mean AUROC (macro): 0.8692
